Exploratory Data Analysis - Jupyter Notebook
================================================

This jupyter notebook contains the results of the Exploratory Data Analysis (EDA).

Ideas are taken from the book:  
Mukhiya, S. K., & Ahmed, U. (2020). Hands-On Exploratory Data Analysis with Python: Perform EDA Techniques to Understand, Summarize, and Investigate Your Data. Packt Publishing.


## Loading the Data

In [171]:
# Operating system functionalities to join pathname components
import os
# Pandas for DataFrame and CSV handling
import pandas as pd

# Join the filepath of the raw data file
filepath = os.path.join("..", "2_data","sickness_table.csv")
print("### Load Data ### Filepath: ", filepath)

# Read the data from CSV file
data_raw = pd.read_csv(filepath)
print("### Load Data ### Structure (first 5 rows): \n", data_raw.head(5))

### Load Data ### Filepath:  ../2_data/sickness_table.csv
### Load Data ### Structure (first 5 rows): 
    Unnamed: 0        date  n_sick   calls  n_duty  n_sby  sby_need  dafted
0           0  2016-04-01      73  8154.0    1700     90       4.0     0.0
1           1  2016-04-02      64  8526.0    1700     90      70.0     0.0
2           2  2016-04-03      68  8088.0    1700     90       0.0     0.0
3           3  2016-04-04      71  7044.0    1700     90       0.0     0.0
4           4  2016-04-05      63  7236.0    1700     90       0.0     0.0


It is therefore structured tiem series data, with the following columns:
- (unnamed): Row number
- date: Date
- n_sick: Number of emergency drivers who have registered a sick call
- calls: Number of emergency calls
- n_duty: Number of emergency drivers on duty
- n_sby: Number of available substitute drivers
- sby_need: Number of substitute drivers to be activated
- dafted: Number of additional duty drivers that have to be activated if the number of on-call drivers are not sufficient

## Data Structure Overview

At first, get an overview about the data structure.

### Data Types

In [172]:
# Get the data types of the columns
print("### Data Overview ### Data types: \n",data_raw.dtypes)

### Data Overview ### Data types: 
 Unnamed: 0      int64
date           object
n_sick          int64
calls         float64
n_duty          int64
n_sby           int64
sby_need      float64
dafted        float64
dtype: object


The columns contain either integer or float values.

## Data Conversions
### Date

The date field is an object, so we need to convert it into a DateTime argument.

In [173]:
# Copy the data for conversion into a new variable
data = data_raw
# Convert the date objects into DateTime
data.date = data.date.apply(lambda x: pd.to_datetime(x,
errors='coerce', utc=True))
print("### Data Conversion ### First 5 rows after convert date: \n", data.head(5))
# Get the data types of the columns again
print("### Data Conversion ### Data types after convert date: \n",data.dtypes)

### Data Conversion ### First 5 rows after convert date: 
    Unnamed: 0                      date  n_sick   calls  n_duty  n_sby  \
0           0 2016-04-01 00:00:00+00:00      73  8154.0    1700     90   
1           1 2016-04-02 00:00:00+00:00      64  8526.0    1700     90   
2           2 2016-04-03 00:00:00+00:00      68  8088.0    1700     90   
3           3 2016-04-04 00:00:00+00:00      71  7044.0    1700     90   
4           4 2016-04-05 00:00:00+00:00      63  7236.0    1700     90   

   sby_need  dafted  
0       4.0     0.0  
1      70.0     0.0  
2       0.0     0.0  
3       0.0     0.0  
4       0.0     0.0  
### Data Conversion ### Data types after convert date: 
 Unnamed: 0                  int64
date          datetime64[ns, UTC]
n_sick                      int64
calls                     float64
n_duty                      int64
n_sby                       int64
sby_need                  float64
dafted                    float64
dtype: object


## Data Cleaning

### Detect missing values

TODO

In [174]:
# Detect missing values (invert the boolean value which indicates not non-missing values within the columns)
~data.notna().any()

Unnamed: 0    False
date          False
n_sick        False
calls         False
n_duty        False
n_sby         False
sby_need      False
dafted        False
dtype: bool

There are no missing values within the dataset.

## Data Analysis

TODO: Gain inside from the data we have

### Averages

In [175]:
#data.mean()
#data.median()
#import matplotlib as plt
#plt.plot(data.date, data.n_sick)